# PSDE project

# 1) IMGT

Browse IMGT to find IGLV, IGKV and IGHV human and mouse locus

- Go to IMGT website : http://www.imgt.org
- In "IMGT Web resources", click on "IMGT Repertoire" (http://www.imgt.org/IMGTrepertoire/)
- Click on part 1 "Locus and genes" (http://www.imgt.org/IMGTrepertoire/LocusGenes/)
- To find the human genes list : click on part 9 "IG genes: Human"
- To find the mouse genes list : click on part 10 "IG genes: Mouse"

For the two last steps, you will get an immunoglobulin and T cell receptor genes coding genes table for human or mouse according you chose part 9 or part 10.

- For each table, handmade, copy/paste in spreadsheet like Excel genes rows that you want (2nd column), here, we will take IGLV, IGKV and IGHV rows.
- In spreadsheet, create a new sheet for human genes and an other one for mouse genes.
- For each sheet, do Ctrl + A, then "Effacer le formatage direct" (french 4ever) (or find a way to convert all entries in text). This step is important !
- Save your Excel (/media/sf_raid/Data/PSDE/IMGT_table.xlsx)

# 2) Manage all genes 

## 2.1) Recover IMGT format

The first step is to download an IGMT file for unique number accession genes existing in spreasheet (7th column)
IMGT file is like Genbank file with IMGT annotation.
We will not donwload an IMGT file directly but the HTML page of IMGT display that we will modify.

Directory creation for human and mouse genes :

In [5]:
### CODE ###
mkdir /media/sf_raid/Data/PSDE/human
mkdir /media/sf_raid/Data/PSDE/mouse

In [6]:
### CODE ###
cd /media/sf_raid/Projects/PSDE
python imgt_api.py > imgt_api.verbose

Content of imgt_api.py :

In [ ]:
### CODE ###
import urllib
import pandas as pd
import sys
reload(sys);
sys.setdefaultencoding("utf8")

##GET IMGT FORMAT FOR ALL GENES ACCESSION NUMBER

#sys.stderr = open('/media/sf_raid/Projects/PSDE/error.txt', 'w+')

df = pd.ExcelFile('/media/sf_raid/Data/PSDE/IMGT_table.xlsx')

df_human = df.parse("human")
df_mouse = df.parse("mouse")

#WARNING : weird space 'sequence(s)for'

#HUMAN
print("HUMAN")
for index, row in df_human.iterrows():
	if str(row[6]) != "" and str(row[6]) != "nan" and str(row[6]) != "Na" and str(row[6]) != "NaN" :
		if len(str(row[6]).split(" "))>1:
			for i in str(row[6]).split(" ") :
				file_name=str(row[1])+"_"+str(i)+'.txt'
				file_name = file_name.replace("/","-")
				print(file_name)
				urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(row[6]), "/media/sf_raid/Data/PSDE/human/"+file_name)
		elif len(str(row[6]).split("\xc2\xa0"))>1:
			for i in str(row[6]).split("\xc2\xa0") :
				file_name=str(row[1])+"_"+str(i)+'.txt'
				file_name = file_name.replace("/","-")
				print(file_name)
				urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(i), "/media/sf_raid/Data/PSDE/human/"+file_name)
		else:
			file_name=str(row[1])+"_"+str(row[6])+'.txt'
			file_name = file_name.replace("/","-")
			print(file_name)
			urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(row[6]), "/media/sf_raid/Data/PSDE/human/"+file_name)
	else:
		print("WARNING : "+str(row[1])+" doesn't have reference sequence in IMGT")

#MOUSE
print("MOUSE")
for index, row in df_mouse.iterrows():
	if str(row[6]) != "" and str(row[6]) != "nan" and str(row[6]) != "Na" and str(row[6]) != "NaN" :
		if len(str(row[6]).split(" "))>1:
			for i in str(row[6]).split(" ") :
				file_name=str(row[1])+"_"+str(i)+'.txt'
				file_name = file_name.replace("/","-")
				print(file_name)
				urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(i), "/media/sf_raid/Data/PSDE/mouse/"+file_name)
		elif len(str(row[6]).split("\xc2\xa0"))>1:
			for i in str(row[6]).split("\xc2\xa0") :
				file_name=str(row[1])+"_"+str(i)+'.txt'
				file_name = file_name.replace("/","-")
				print(file_name)
				urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(i), "/media/sf_raid/Data/PSDE/mouse/"+file_name)
		else:
			file_name=str(row[1])+"_"+str(row[6])+'.txt'
			file_name = file_name.replace("/","-")
			print(file_name)
			urllib.urlretrieve("http://www.imgt.org/ligmdb/view?format=IMGT&id="+str(row[6]), "/media/sf_raid/Data/PSDE/mouse/"+file_name)
	else:
		print("WARNING : "+str(row[1])+" doesn't have reference sequence in IMGT")

## 2.2) HTML modifications

Then, once the HTML files are recovered, we chop each file to create a GenBank file like.

In [7]:
### CODE ###
cd /media/sf_raid/Projects/PSDE
python imgt_manage_file.py > imgt_manage.verbose

Content of imgt_manage_file.py :

In [ ]:
### CODE ###
import urllib
import pandas as pd
import sys
from os import listdir
from os.path import isfile, join

##HTML MODIFICATIONS

files = [f for f in listdir("/media/sf_raid/Data/PSDE/human/") if isfile(join("/media/sf_raid/Data/PSDE/human/", f))]
for i in files :
	print(i)
	bool_write=False
	fr = open("/media/sf_raid/Data/PSDE/human/"+i,"r")
	fw = open("/media/sf_raid/Data/PSDE/human/"+i[:-4]+"_imgt.txt","w")
	for line in fr :
		if line.startswith("<pre>"):
			bool_write=True
			line=line[5:]
		if line.startswith("</pre>"):
			break
		if bool_write:
			fw.write(line)
	fw.close()

files = [f for f in listdir("/media/sf_raid/Data/PSDE/mouse/") if isfile(join("/media/sf_raid/Data/PSDE/mouse/", f))]
for i in files :
	print(i)
	bool_write=False
	fr = open("/media/sf_raid/Data/PSDE/mouse/"+i,"r")
	fw = open("/media/sf_raid/Data/PSDE/mouse/"+i[:-4]+"_imgt.txt","w")
	for line in fr :
		if line.startswith("<pre>"):
			bool_write=True
			line=line[5:]
		if line.startswith("</pre>"):
			break
		if bool_write:
			fw.write(line)
	fw.close()

# 2.3) Sequence manipulations

For each file downloaded, we collect V-GENE, L-PART1, V-EXON and the sequence belongs to the L-PART1 + V-EXON. All results are sent to a new file in fasta format with V-GENE as gene sequence.

In [ ]:
### CODE ###
cd /media/sf_raid/Projects/PSDE
python imgt_get_infos.py > imgt_get_infos.verbose

Content of imgt_get_infos.py :

In [ ]:
### CODE ###
import urllib
import pandas as pd
import sys
from os import listdir
from os.path import isfile, join
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

type_file = "mouse"

files = [f for f in listdir("/media/sf_raid/Data/PSDE/"+type_file+"/") if isfile(join("/media/sf_raid/Data/PSDE/"+type_file+"/", f))]
#files = ["10_imgt.txt"]
#files = ["test_imgt.txt"]
#files = ["181_imgt.txt"]
#files = ["47_imgt.txt", "168_imgt.txt"]

total_count=0

with open("/media/sf_raid/Data/PSDE/"+type_file+".fasta", "w") as output_handle:
	for i in files :
		FT=[]
		FT_seq=[]
		SQ=[]
		seq=""
		L1L2V_tab=[]
		L1L2V_dict={}
		L1L2V_dict_final={}
		if str(i[-8:]) == "imgt.txt":
			print(i)
			f = open("/media/sf_raid/Data/PSDE/"+type_file+"/"+i,"r")
			boolSQ=False
			for line in f :
				if line.split("  ")[0] == "FT" :
					line = filter(None, line.split(" "))
					line[-1]=line[-1][0:-1]
					if len(line) > 3 :
						line[1] = ' '.join(line[1:])
						del line[2:]
					elif len(line) == 3 :
						if ".." not in line[-1]:
							line[1] = ' '.join(line[1:])
							del line[2:]
					FT.append(line)
				elif line.split("  ")[0] == "//" :
					boolSQ=False
				elif line.split("  ")[0] == "SQ" :
					boolSQ=True
				if boolSQ==True and line.split("  ")[0] == "":
					for j in line.split(" "):
						if j.startswith("a") or j.startswith("t") or j.startswith("c") or j.startswith("g") or j.startswith("A") or j.startswith("T") or j.startswith("C") or j.startswith("G") :
							seq+=str(j)

			boolGene=False
			check_gene=False
			check_lpart1=False
			check_vexon=False
			for j in FT:
				if boolGene :
					if j[1] == "/partial" or j[1] == "/complement" or '&' in FT_seq[-1][-1]:
						#print("DEGAGE  : bad name or position")
						check_gene=False
						boolGene=False
						del FT_seq[-1]
					else :
						#print(j)
						FT_seq[-1].append(j[1])
						boolGene=False
				if len(j) == 3:
					if j[1] == "V-GENE" :
						if check_gene and not check_lpart1 and not check_vexon :
							#print("DEGAGE no lpart and exon")
							del FT_seq[-1]
						elif check_gene and not check_lpart1 and check_vexon :
							#print("DEGAGE no lpart1")
							del FT_seq[-1]
							del FT_seq[-1]
						elif check_gene and check_lpart1 and not check_vexon :
							#print("DEGAGE no vexon")
							del FT_seq[-1]
							del FT_seq[-1]
						#print("\n")
						#print(FT_seq)
						#print("----------------------")
						#print("V-GENE")

						check_gene=True
						check_lpart1=False
						check_vexon=False
						boolGene=True

						FT_seq.append(j)
					elif j[1] == "L-PART1" :
						if check_gene and '&' not in j[-1]:
							#print("L-PART1")
							FT_seq.append(j)
							check_lpart1=True
						
					elif j[1] == "V-EXON" :
						if check_gene and '&' not in j[-1]:
							#print("V-EXON")
							FT_seq.append(j)
							check_vexon=True
				
			#print(FT_seq)
			index=0
			for j in FT_seq	:
				FT_seq[index].append(seq[int(FT_seq[index][2].split("..")[0])-1:int(FT_seq[index][2].split("..")[1])])
				index+=1

			index=""
			for j in FT_seq	:
				if j[1] == "V-GENE" :
					index=str(j[3].split("&quot;")[1])
					#print("Got a V-GENE : "+str(index))
					L1L2V_dict[index] = []
				elif j[1] == "L-PART1" :
					L1L2V_dict[index].append(j[-1])
				elif j[1] == "V-EXON" :
					L1L2V_dict[index].append(j[-1])

			count=0
			for key, value in L1L2V_dict.items():
				if len(value) == 2 :
					if key not in L1L2V_dict_final :
						count+=1
						L1L2V_dict_final[key]=''.join(str(k) for k in value)
					#else :
					#	print("Already in here !")

			print(str(count)+" entries added to the dictionnary")
			total_count+=count

			for key, value in L1L2V_dict_final.items():
				record = SeqRecord(Seq(str(value), generic_dna), id=str(key), name="", description="")
				SeqIO.write(record, output_handle, "fasta")
print(str(total_count)+" entries")

# 2.4) Predoss

Use of Predoss for the prediction of donor splice sites, mainly in vertebrates, in eukaryotic gene.
Artificial Neural Network used.

http://cabgrid.res.in:8080/predoss/

Manually, send your files to Predoss and run using ANN method.
Then download the result file in .tsv

# 2.5) Filtering

## 2.5.1) Detection of donor splice sites in phase

To check the phasing of all result row. Just create a new column and for each row do : (Start_position-2)/3.
Then, create an other column next to the previous one and check if the previous result is integer or not.
Keep the true result and here we are.

To end the filtering session, just filter on gene name and score decreasing.